## Comparing Performance of Pareto-Fronts

Comparing performance of Pareto-Fronts constructed using:
- Reference Bitrate Ladder: Exhaustive Encoding
- Cross-Over Bitrates: From target Cross-Over bitrates used during training of models.
- Cross-Over Bitrates: From target Cross-Over VMAFs used during training of models.

against fixed bitrate ladder

### Imports

Libraries

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import os, sys, warnings
import pickle
from tqdm import tqdm
warnings.filterwarnings("ignore")
sys.path.append("/home/krishna/Constructing-Per-Shot-Bitrate-Ladders-using-Visual-Information-Fidelity")
import functions.IO_functions as IO_functions
import functions.extract_functions as extract_functions
import functions.standard_bitrate_ladder_functions as standard_bitrate_ladder_functions
import bitrate_ladder_construction.BL_functions.cross_over_rate_control as cross_over_rate_control
import bitrate_ladder_construction.BL_functions.bitrate_ladder_functions as bitrate_ladder_functions
import quality_ladder_construction.QL_functions.quality_ladder_functions as quality_ladder_functions
import quality_ladder_construction.QL_functions.cross_over_quality as cross_over_quality
import defaults

### Parameters

In [ ]:
video_files = defaults.Test_Video_Titles

### Constructing Apple's Fixed Bitrate Ladder

In [ ]:
standard_bitrate_ladder_functions.Construct_Fixed_Bitrate_Ladder()

### Constructing Reference Bitrate Ladder

In [ ]:
standard_bitrate_ladder_functions.Construct_Reference_Bitrate_Ladder(
	codec="libx265",
	preset="medium",
	video_files=video_files
)

In [ ]:
Metrics = []
for video_file in video_files:
	Metrics.append(bitrate_ladder_functions.Calculate_BD_metrics(
		video_file=video_file,
		codec="libx265",
		preset="medium",
		bitrate_ladder_path="/home/krishna/Constructing-Per-Shot-Bitrate-Ladders-using-Visual-Information-Fidelity/bitrate_ladder_construction/bitrate_ladders/standard/reference_bitrate_ladder.npy",
		fixed_bitrate_ladder_path="/home/krishna/Constructing-Per-Shot-Bitrate-Ladders-using-Visual-Information-Fidelity/bitrate_ladder_construction/bitrate_ladders/standard/fixed_bitrate_ladder.npy",
		reference_bitrate_ladder_path="/home/krishna/Constructing-Per-Shot-Bitrate-Ladders-using-Visual-Information-Fidelity/bitrate_ladder_construction/bitrate_ladders/standard/reference_bitrate_ladder.npy"
	))

Metrics = np.asarray(Metrics)
Metrics = Metrics[np.logical_not(np.all(np.isnan(Metrics), axis=1)), :]
mean = np.round(np.mean(Metrics, axis=0), decimals=3)
std = np.round(np.std(Metrics, axis=0), decimals=3)

# Histogram plot of BD-metrics
plt.figure(figsize=(12,8))

plt.subplot(2,2,1)
plt.title(r"BD-Rate wrt AL ($\mu$={}, $\sigma$={})".format(mean[0], std[0]))
plt.grid()
sns.histplot(data=Metrics[:,0], bins=20, kde=True, element="step")

plt.subplot(2,2,2)
plt.title(r"BD-VMAF wrt AL ($\mu$={}, $\sigma$={})".format(mean[1], std[1]))
plt.grid()
sns.histplot(data=Metrics[:,1], bins=20, kde=True, element="step")

plt.subplot(2,2,3)
plt.title(r"BD-Rate wrt RL ($\mu$={}, $\sigma$={})".format(mean[2], std[2])) 
plt.grid()
sns.histplot(data=Metrics[:,2], bins=20, kde=True, element="step")

plt.subplot(2,2,4)
plt.title(r"BD-VMAF wrt RL ($\mu$={}, $\sigma$={})".format(mean[3], std[3]))
plt.grid()
sns.histplot(data=Metrics[:,3], bins=20, kde=True, element="step")

plt.show()

### Constructing True Cross-Over Bitrate Ladder

In [ ]:
Bitrate_Ladders = {}
for video_file in video_files:
	video_rq_points_info = IO_functions.read_create_jsonfile(os.path.join(defaults.rq_points_dataset_path, "libx265", "medium", video_file, "crfs.json"))
	RQ_pairs = extract_functions.Extract_RQ_Information(
		video_rq_points_info=video_rq_points_info,
		quality_metric="vmaf",
		resolutions=defaults.resolutions,
		CRFs=defaults.CRFs,
		bitrates=None,
		QPs=None,
		min_quality=defaults.min_quality,
		max_quality=defaults.max_quality,
		min_bitrate=defaults.min_bitrate,
		max_bitrate=defaults.max_bitrate,
		set_bitrate_log_base=2
	)

	CrossOver_Bitrates = cross_over_rate_control.CrossOver_Bitrates(RQ_pairs=RQ_pairs, Resolutions=defaults.resolutions)

	Bitrate_Ladder = {}
	for i in range(len(defaults.evaluation_bitrates)):
		# Switching happens to higher resolution when bitrate >= crossover_bitrate of corresponding higher resolution.
		b = defaults.evaluation_bitrates[i]
		Bitrate_Ladder[b] = None

		for j in range(1+len(CrossOver_Bitrates)):
			if (j==0) and (b >= CrossOver_Bitrates[j]):
				Bitrate_Ladder[b] = defaults.resolutions[0]
			elif (j <= len(CrossOver_Bitrates)-1) and (CrossOver_Bitrates[j] <= b < CrossOver_Bitrates[j-1]):
				Bitrate_Ladder[b] = defaults.resolutions[j]
			elif (j==len(CrossOver_Bitrates)) and (b < CrossOver_Bitrates[j-1]):
				Bitrate_Ladder[b] = defaults.resolutions[-1]
			else:
				None

		if Bitrate_Ladder[b] is None:
			assert False, "Something is Wrong"

	Bitrate_Ladders[video_file] = Bitrate_Ladder

np.save("/home/krishna/Constructing-Per-Shot-Bitrate-Ladders-using-Visual-Information-Fidelity/bitrate_ladder_construction/bitrate_ladders/standard/true_crossover_bitrate_ladder.npy", Bitrate_Ladders)

In [ ]:
Metrics = []
for video_file in video_files:
	Metrics.append(bitrate_ladder_functions.Calculate_BD_metrics(
		video_file=video_file,
		codec="libx265",
		preset="medium",
		bitrate_ladder_path="/home/krishna/Constructing-Per-Shot-Bitrate-Ladders-using-Visual-Information-Fidelity/bitrate_ladder_construction/bitrate_ladders/standard/true_crossover_bitrate_ladder.npy",
		fixed_bitrate_ladder_path="/home/krishna/Constructing-Per-Shot-Bitrate-Ladders-using-Visual-Information-Fidelity/bitrate_ladder_construction/bitrate_ladders/standard/fixed_bitrate_ladder.npy",
		reference_bitrate_ladder_path="/home/krishna/Constructing-Per-Shot-Bitrate-Ladders-using-Visual-Information-Fidelity/bitrate_ladder_construction/bitrate_ladders/standard/reference_bitrate_ladder.npy"
	))
	
Metrics = np.asarray(Metrics)
Metrics = Metrics[np.logical_not(np.all(np.isnan(Metrics), axis=1)), :]
mean = np.round(np.mean(Metrics, axis=0), decimals=3)
std = np.round(np.std(Metrics, axis=0), decimals=3)

# Histogram plot of BD-metrics
plt.figure(figsize=(12,8))

plt.subplot(2,2,1)
plt.title(r"BD-Rate wrt AL ($\mu$={}, $\sigma$={})".format(mean[0], std[0]))
plt.grid()
sns.histplot(data=Metrics[:,0], bins=20, kde=True, element="step")

plt.subplot(2,2,2)
plt.title(r"BD-VMAF wrt AL ($\mu$={}, $\sigma$={})".format(mean[1], std[1]))
plt.grid()
sns.histplot(data=Metrics[:,1], bins=20, kde=True, element="step")

plt.subplot(2,2,3)
plt.title(r"BD-Rate wrt RL ($\mu$={}, $\sigma$={})".format(mean[2], std[2]))
plt.grid()
sns.histplot(data=Metrics[:,2], bins=20, kde=True, element="step")

plt.subplot(2,2,4)
plt.title(r"BD-VMAF wrt RL ($\mu$={}, $\sigma$={})".format(mean[3], std[3]))
plt.grid()
sns.histplot(data=Metrics[:,3], bins=20, kde=True, element="step")

plt.show()

### Constructing True Cross-Over Quality Ladder

In [ ]:
Quality_Ladders = {}
for video_file in video_files:
	video_rq_points_info = IO_functions.read_create_jsonfile(os.path.join(defaults.rq_points_dataset_path, "libx265", "medium", video_file, "crfs.json"))
	RQ_pairs = extract_functions.Extract_RQ_Information(
		video_rq_points_info=video_rq_points_info,
		quality_metric="vmaf",
		resolutions=defaults.resolutions,
		CRFs=defaults.CRFs,
		bitrates=None,
		QPs=None,
		min_quality=defaults.min_quality,
		max_quality=defaults.max_quality,
		min_bitrate=defaults.min_bitrate,
		max_bitrate=defaults.max_bitrate,
		set_bitrate_log_base=2
	)

	CrossOver_Qualities = cross_over_quality.CrossOver_Quality(RQ_pairs=RQ_pairs, Resolutions=defaults.resolutions)
	CrossOver_Qualities = np.asarray(CrossOver_Qualities)/100.0


	Quality_Ladder = {}
	for i in range(len(defaults.evaluation_qualities)):
		# Switching happens to higher resolution when quality >= crossover_quality of corresponding higher resolution.
		q = defaults.evaluation_qualities[i]
		Quality_Ladder[q] = None
		
		for j in range(1+len(CrossOver_Qualities)):
			if (j==0) and (q >= CrossOver_Qualities[j]):
				Quality_Ladder[q] = defaults.resolutions[0]
			elif (j <= len(CrossOver_Qualities)-1) and (CrossOver_Qualities[j] <= q < CrossOver_Qualities[j-1]):
				Quality_Ladder[q] = defaults.resolutions[j]
			elif (j==len(CrossOver_Qualities)) and (q < CrossOver_Qualities[j-1]):
				Quality_Ladder[q] = defaults.resolutions[-1]
			else:
				None

		if Quality_Ladder[q] is None:
			assert False, "Something is Wrong"

	Quality_Ladders[video_file] = Quality_Ladder

np.save("/home/krishna/Constructing-Per-Shot-Bitrate-Ladders-using-Visual-Information-Fidelity/bitrate_ladder_construction/bitrate_ladders/standard/true_crossover_quality_ladder.npy", Quality_Ladders)

In [ ]:
Metrics = []
for video_file in video_files:
	Metrics.append(quality_ladder_functions.Calculate_BD_metrics(
		video_file=video_file,
		codec="libx265",
		preset="medium",
		quality_ladder_path="/home/krishna/Constructing-Per-Shot-Bitrate-Ladders-using-Visual-Information-Fidelity/bitrate_ladder_construction/bitrate_ladders/standard/true_crossover_quality_ladder.npy",
		fixed_bitrate_ladder_path="/home/krishna/Constructing-Per-Shot-Bitrate-Ladders-using-Visual-Information-Fidelity/bitrate_ladder_construction/bitrate_ladders/standard/fixed_bitrate_ladder.npy",
		reference_bitrate_ladder_path="/home/krishna/Constructing-Per-Shot-Bitrate-Ladders-using-Visual-Information-Fidelity/bitrate_ladder_construction/bitrate_ladders/standard/reference_bitrate_ladder.npy"
	))

Metrics = np.asarray(Metrics)
Metrics = Metrics[np.logical_not(np.all(np.isnan(Metrics), axis=1)), :]
mean = np.round(np.mean(Metrics, axis=0), decimals=3)
std = np.round(np.std(Metrics, axis=0), decimals=3)

# Histogram plot of BD-metrics
plt.figure(figsize=(12,8))

plt.subplot(2,2,1)
plt.title(r"BD-Rate wrt AL ($\mu$={}, $\sigma$={})".format(mean[0], std[0]))
plt.grid()
sns.histplot(data=Metrics[:,0], bins=20, kde=True, element="step")

plt.subplot(2,2,2)
plt.title(r"BD-VMAF wrt AL ($\mu$={}, $\sigma$={})".format(mean[1], std[1]))
plt.grid()
sns.histplot(data=Metrics[:,1], bins=20, kde=True, element="step")

plt.subplot(2,2,3)
plt.title(r"BD-Rate wrt RL ($\mu$={}, $\sigma$={})".format(mean[2], std[2]))
plt.grid()
sns.histplot(data=Metrics[:,2], bins=20, kde=True, element="step")

plt.subplot(2,2,4)
plt.title(r"BD-VMAF wrt RL ($\mu$={}, $\sigma$={})".format(mean[3], std[3]))
plt.grid()
sns.histplot(data=Metrics[:,3], bins=20, kde=True, element="step")

plt.show()